In [1]:
%matplotlib inline
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import utils
import importlib
importlib.reload(utils)
from utils import *

In [2]:
import re, string
import pandas as pd
import numpy as np
import csv
import xlsxwriter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
from sklearn.model_selection import KFold 
from sklearn.model_selection import RepeatedKFold


In [3]:
df = pd.read_excel("file skripsi\\Data Training RS V10.xlsx")
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)


In [4]:
positif = df.loc[df['sent'] == 1, 'Review'].copy().reset_index(drop=True)
netral = df.loc[df['sent'] == 0, 'Review'].copy().reset_index(drop=True)
negatif = df.loc[df['sent'] == -1, 'Review'].copy().reset_index(drop=True)

In [5]:
len(positif), len(netral), len(negatif)

(2771, 2788, 2738)

In [6]:
vect = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['Review'].values, df['sent'].values, test_size=0.2, random_state=0)

In [8]:
tfidf_train = vect.fit_transform(X_train)
tfidf_test = vect.transform(X_test)
print(tfidf_train)

  (0, 1066)	0.5837152637700084
  (0, 430)	0.8119584292572555
  (2, 1383)	0.5452791055425255
  (2, 1471)	0.3320611429023645
  (2, 480)	0.3457369860281216
  (2, 410)	0.45418564011746254
  (2, 1167)	0.3457369860281216
  (2, 956)	0.3834753860791987
  (3, 926)	0.22558147419715285
  (3, 85)	0.6325321595552098
  (3, 767)	0.563067297631591
  (3, 911)	0.29421485721133483
  (3, 198)	0.38132519161659506
  (4, 926)	0.09094022091016112
  (4, 207)	0.17994425411316706
  (4, 1158)	0.16447775865012323
  (4, 498)	0.2553355485335449
  (4, 720)	0.19846586154213533
  (4, 924)	0.25057463669383984
  (4, 1083)	0.25057463669383984
  (4, 669)	0.47844105310956575
  (4, 438)	0.15103144798064194
  (4, 552)	0.404559664656427
  (4, 1365)	0.1689270335067535
  (4, 1233)	0.27528802096794347
  :	:
  (6634, 926)	0.08256492078364698
  (6634, 911)	0.21537075653469123
  (6634, 340)	0.10396163894540267
  (6634, 1028)	0.2883193076392715
  (6634, 962)	0.13315804046475085
  (6634, 434)	0.1994973225475217
  (6634, 750)	0.1801877

In [10]:
nb = MultinomialNB()
nb.fit(tfidf_train, y_train)
preds = nb.predict(tfidf_test)


In [11]:
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 
# n_split 5 ini aku masih ragu harusnya 5 atau 10 gitu
nb = MultinomialNB()
X = df['Review']
Y = df['sent']
scores = []
for i in range (10):    
    result = next(kf.split(X), None)
    x_train = X.iloc[result[0]]
    x_test = X.iloc[result[1]]
    y_train = Y.iloc[result[0]]
    y_test = Y.iloc[result[0]]
    tfidf_xtrain = vect.fit_transform(x_train)
    tfidf_xtest = vect.transform(x_test)
    model = nb.fit(tfidf_xtrain,y_train)
    predictions = nb.predict(tfidf_xtest)
    scores.append(model.score(tfidf_xtrain,y_train))
print('Scores from each Iteration : ', scores)
print('Average K-fold Score : ', np.mean(scores))

Scores from each Iteration :  [0.8202501130028628, 0.8151273165586862, 0.8219074883230375, 0.8130179297875546, 0.8169353623625132, 0.8074431218924213, 0.8139219526894681, 0.8188940786499924, 0.8191954196172969, 0.8199487720355583]
Average K-fold Score :  0.816664155491939


In [12]:
def konten(kata):
    fo = pd.read_excel(kata)
    rev = fo['Review']
    rev = rev.values.tolist()
#     texts = []
#     sentiments = []
#     probability = []
    item = []
    for kalimat in rev:
        it = []
        kata = sent_tokenize(kalimat)
        tf_train = vect.fit_transform(X_train)
        tf_test = vect.transform(kata)
        nb = MultinomialNB()
        nb.fit(tf_train, y_train)
        preds = nb.predict(tf_test)

        if preds == -1:
            pr3 = nb.predict_proba(tf_test)
            pr3 = pr3.reshape(-1, 1).tolist()
            it.append(kalimat)
            it.append('negatif')
            it.append(pr3[0])

        elif preds  == 0:
            pr3 = nb.predict_proba(tf_test)
            pr3 = pr3.reshape(-1, 1).tolist()
            it.append(kalimat)
            it.append('netral')
            it.append(pr3[1])
        elif preds == 1:
            pr3 = nb.predict_proba(tf_test)
            pr3 = pr3.reshape(-1, 1).tolist()        
            it.append(kalimat)
            it.append('positif')
            it.append(pr3[2])
#             trainDF = pd.DataFrame()
#             trainDF['text'] = texts
#             trainDF['sentiment'] = sentiments
#             trainDF['probabilitas'] = probability
        item.append(it)

    return item


In [27]:
cb = konten('E:\\kuliah\\Pengolahan Bahasa Alami\\Bismillah Skripsi Baru\\file skripsi\\Data Testing Preprocessing v2.xlsx')
print(cb)

[['pelayanan bagus bertanya pendaftaran lebih mudah', 'positif', [0.3756198772880598]]]


In [96]:
import csv
fileurl = 'file skripsi\Sentiment Analysis v10 net.csv'

with open(fileurl, "w",encoding='utf-8', newline ='') as f:
    writer = csv.writer(f)
    writer.writerows(cb)
    
    
print('exported: '+fileurl)
print('-Selesai-')

exported: file skripsi\Sentiment Analysis v10 pos.csv
-Selesai-


In [30]:
df1 = pd.read_excel("file skripsi\Sentiment Analysis fix v10.xlsx")
df1.replace('', np.nan, inplace=True)
df1.dropna(inplace=True)

In [31]:
X= df1['Review'].values
Y = df1['sent'].values
X_trainp, X_testp, Y_trainp, Y_testp = train_test_split(X, Y, test_size=0.3, random_state=42)
len(X)

5268